Universidad del Valle de Guatemala
<br>
Inteligencia Artificial
<br>
Andrea Argüello 17801
<br>
<center><b><font size="6">Laboratorio 2: Redes Neuronales</font></b></center>

In [1]:
import numpy as np
import pandas as pd
import neural_networks as nn
from scipy.optimize import minimize

In [2]:
NORMALIZE = 1000.0
#df = pd.read_csv(r'./datasets/csv/fashion-mnist_train.csv')
test = pd.read_csv(r'./datasets/csv/fashion-mnist_test.csv')

#shuffled = df.sample(frac=1)

#train, validate = np.split(shuffled, [50000])
#train.to_csv(path_or_buf='./datasets/csv/fashion-mnist_train_50k.csv',index=False)
#validate.to_csv(path_or_buf='./datasets/csv/fashion-mnist_validate.csv',index=False)

train = pd.read_csv(r'./datasets/csv/fashion-mnist_train_50k.csv')
validate = pd.read_csv(r'./datasets/csv/fashion-mnist_validate.csv')

train_X = np.asarray(train[train.columns[~train.columns.isin(['label'])]]) / NORMALIZE
train_y = np.asarray(train[['label']])
validate_X = np.asarray(validate[validate.columns[~validate.columns.isin(['label'])]]) / NORMALIZE
validate_y = (validate[['label']])
test_X = np.asarray(test[test.columns[~test.columns.isin(['label'])]]) / NORMALIZE
test_y = np.asarray(test[['label']])

In [3]:
# A setear variables
NETWORK_ARCH = np.array([
    784,
    90, # ~(784*10)**0.5
    10
    ])
theta_shapes = np.hstack((
    NETWORK_ARCH[1:].reshape(len(NETWORK_ARCH)-1,1),
    (NETWORK_ARCH[:-1]+1).reshape(len(NETWORK_ARCH)-1,1)
))

In [4]:
# Otras variables
#flat_thetas = nn.flatten_list_of_arrays([
#    np.random.rand(*theta_shape) for theta_shape in theta_shapes
#])

In [5]:
# np.save('flat_thetas', flat_thetas)

In [6]:
flat_thetas = np.load('flat_thetas.npy')

# Optimización con scipy.optimize.minimize

In [7]:
result = minimize(
    fun=nn.cost_function,
    x0=flat_thetas,
    args=(theta_shapes,train_X,train_y),
    method='L-BFGS-B',
    jac=nn.back_propagation,
    options={'disp':True, 'maxiter':3000}
)

/home/ranchobojon/Documents/SEMESTRE_1_2020/AI/NeuralNetworks/neural_networks.py:43: RuntimeWarning: divide by zero encountered in log
  return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)
/home/ranchobojon/Documents/SEMESTRE_1_2020/AI/NeuralNetworks/neural_networks.py:43: RuntimeWarning: invalid value encountered in multiply
  return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)
/home/ranchobojon/Documents/SEMESTRE_1_2020/AI/NeuralNetworks/.venv/lib/python3.6/site-packages/numpy/core/_methods.py:38: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [8]:
print(result)

      fun: nan
 hess_inv: <71560x71560 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.90574595e-04, -6.51877179e-12, -5.97417539e-12, ...,
       -3.50509011e+00, -3.50503047e+00, -3.50506232e+00])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 21
      nit: 0
   status: 2
  success: False
        x: array([0.96064244, 0.30868447, 0.53527957, ..., 0.0950575 , 0.06972063,
       0.44905068])
